In [ ]:
# Encoder/decoder transformer, like in the paper

# Single Encoder block

# Embed input into a length 512 vector
# Add positional encoding to the input
# Alternating sin and cosine functions across the embedding dimension (512)
# i = position in the embedding dimension
#PE(pos,2i) = sin(pos/100002i/dmodel )
#PE(pos,2i+1) = cos(pos/100002i/dmodel
# Sum positional encoding and input
# apply dropout with chance .1

# Self-attention
# Each token in the input sequence is a query
# Every other token is a key
# Project query and key into a new dimensional space with a linear transform (this is multi-head attention)
# We do the following 16 times for multi-head attention
# Dot the projects query with the projected key matrix
# QK
# Attention(Q, K, V ) = softmax( QKT / √dk)V
# √dk is the square root of the value dimension
# Keys and values are the same in this particular case
# Basically, compute an attention score (scalar) between each query and each key, then scale the value by that number
# So less relevant other tokens are minimized
# Concat the results of the attention equation
# Run through another linear layer to reproject
# Each attention head outputs 1/16th of the input embedding len

# After doing multi-head attention (16)
# Apply dropout with chance .1
# Add the input to the layer (original embedded sequences) and the output of attention
# Run layer normalization (unclear which layer norm to use)

# Run a feed forward network
# Add input to the layer to the output of the ff network
# Normalize again

# Single decoder block

# Shift outputs right, to start with start token
# Do positional encoding
# Do dropout with chance .1
# Mask outputs, so queries can only see keys that came before the query
# Run multi-head attention
# Add and norm

# Run multi-head attention again, but this time v,k is from encoder stack, and q is from decoder stack
# Apply dropout with chance .1
# When doing add and norm, add in the decoder stack input

# Feed forward

# At top of stack, do another linear layer and softmax

# Might want to move layer norm inside the residual block - https://arxiv.org/pdf/2002.04745.pdf
# Layer normalization - https://arxiv.org/pdf/1607.06450.pdf

In [53]:
import numpy as np
import torch
from torch import nn
import functorch
import sys
import os
import math
sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 1
SP_VOCAB_SIZE = 1000
TRAIN_SIZE = 500

In [314]:
from text_data import CNNDatasetWrapper

class Wrapper(CNNDatasetWrapper):
    split_lengths = [TRAIN_SIZE, math.floor(TRAIN_SIZE * .1), 100]
    x_length = 15
    target_length = 14

wrapper = Wrapper(SP_VOCAB_SIZE, DEVICE)

datasets = wrapper.generate_datasets(BATCH_SIZE)
train = datasets["train"]
valid = datasets["validation"]

Found cached dataset cnn_dailymail (/Users/vik/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
100%|██████████| 3/3 [00:00<00:00, 43.66it/s]
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tokens.txt --model_prefix=cnn_dailymail --vocab_size=1000 --model_type=unigram
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokens.txt
  input_format: 
  model_prefix: cnn_dailymail
  model_type: UNIGRAM
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piec

In [315]:
class MultiHeadAttention(nn.Module):
    def __init__(self, input_units, attention_heads, mask=False):
        super(MultiHeadAttention, self).__init__()
        self.input_units = input_units
        self.attention_heads = attention_heads
        self.head_units = int(input_units/attention_heads)
        self.mask = mask
        if self.head_units * self.attention_heads != self.input_units:
            raise Exception("Invalid input units and heads combo")

        k = math.sqrt(1/self.head_units)
        self.query_weights = nn.Parameter(torch.rand(self.attention_heads, input_units, self.head_units) * 2 * k - k)
        self.key_weights = nn.Parameter(torch.rand(self.attention_heads, input_units, self.head_units) * 2 * k - k)
        self.value_weights = nn.Parameter(torch.rand(self.attention_heads, input_units, self.head_units) * 2 * k - k)

    def forward(self, queries, context):
        exp_queries = queries.repeat(self.attention_heads, 1, 1)
        exp_context = context.repeat(self.attention_heads, 1, 1)
        queries = torch.bmm(exp_queries, self.query_weights)
        keys = torch.bmm(exp_context, self.key_weights)
        values = torch.bmm(exp_context, self.value_weights)

        # Sequence-wise softmax, so attention between one sequence and other sequences sums to 1
        attention = torch.bmm(queries, keys.swapaxes(1,2)) / np.sqrt(self.head_units)
        if self.mask:
            # Prevent decoder queries from looking at tokens that come after
            # Do this by setting attention to negative infinity, so it is softmaxed to zero in the next step
            mask = torch.zeros((attention.shape[-2], attention.shape[-1]))
            mask_indices = np.triu_indices(attention.shape[-2], k=1, m=attention.shape[-1])
            mask[mask_indices] = -torch.inf
            attention += mask

        attention = torch.softmax(attention, dim=2)
        weighted_values = torch.bmm(attention, values)
        weighted_values = weighted_values.swapaxes(0,1).reshape(x.shape[0], -1)
        return weighted_values

In [316]:
class EncoderBlock(nn.Module):
    def __init__(self, input_units, attention_heads, hidden_units=2048):
        super(EncoderBlock, self).__init__()
        self.input_units = input_units
        self.attention_heads = attention_heads
        self.hidden_units = hidden_units

        self.mha = MultiHeadAttention(self.input_units, self.attention_heads)
        self.dropout = nn.Dropout(.1)
        self.linear1 = nn.Linear(self.input_units, hidden_units)
        self.linear2 = nn.Linear(hidden_units, self.input_units)
        self.relu = nn.ReLU()
        self.lns = nn.ModuleList(nn.LayerNorm(self.input_units) for _ in range(2))

    def forward(self, x):
        weighted_values = self.dropout(self.mha(x, x))
        attn_output = self.lns[0](x + weighted_values)

        reprojected = self.dropout(self.linear2(self.relu(self.linear1(attn_output))))
        block_output = self.lns[1](attn_output + reprojected)

        return block_output

In [327]:
class DecoderBlock(nn.Module):
    def __init__(self, input_units, attention_heads, hidden_units=2048):
        super(DecoderBlock, self).__init__()
        self.input_units = input_units
        self.attention_heads = attention_heads
        self.hidden_units = hidden_units

        self.in_attn = MultiHeadAttention(self.input_units, self.attention_heads, mask=True)
        self.context_attn = MultiHeadAttention(self.input_units, self.attention_heads)
        self.dropout = nn.Dropout(.1)
        self.linear1 = nn.Linear(self.input_units, hidden_units)
        self.linear2 = nn.Linear(hidden_units, self.input_units)
        self.relu = nn.ReLU()
        self.lns = nn.ModuleList(nn.LayerNorm(self.input_units) for _ in range(3))

    def forward(self, queries, context):
        weighted_values = self.dropout(self.in_attn(queries, queries))
        attn_output = self.lns[0](queries + weighted_values)

        decoder_values = self.dropout(self.context_attn(attn_output, context))
        decoder_output = self.lns[1](attn_output + decoder_values)

        reprojected = self.dropout(self.linear2(self.relu(self.linear1(decoder_output))))
        block_output = self.lns[2](decoder_output + reprojected)
        return block_output

In [328]:
class Transformer(nn.Module):
    def __init__(self, input_units, hidden_units, attention_heads, blocks=2):
        super(Transformer, self).__init__()
        self.input_units = input_units
        self.hidden_units = hidden_units
        self.attention_heads = attention_heads
        self.blocks = blocks

        k = math.sqrt(1/self.hidden_units)
        self.embedding = nn.Embedding(self.input_units, self.hidden_units)
        self.dropout = nn.Dropout(.1)
        self.encoders = nn.ModuleList(EncoderBlock(hidden_units, attention_heads) for _ in range(self.blocks))
        self.decoders = nn.ModuleList(DecoderBlock(hidden_units, attention_heads) for _ in range(self.blocks))
        self.output_embedding = nn.Parameter(torch.rand(hidden_units, self.input_units) * 2 * k - k)

    def forward(self, x, y):
        embedded = self.embedding(x)
        pos_encoding = torch.from_numpy(self.encoding(embedded.shape[0], self.hidden_units))
        network_in = self.dropout(embedded + pos_encoding)
        enc_outputs = network_in.unsqueeze(0)

        for i in range(self.blocks):
            block_output = self.encoders[i](enc_outputs[i])
            enc_outputs = torch.cat((enc_outputs, block_output.unsqueeze(0)), dim=0)


        dec_outputs = self.embedding(y).unsqueeze(0)
        context = enc_outputs[-1]
        for i in range(self.blocks):
            block_output = self.decoders[i](dec_outputs[i], context)
            dec_outputs = torch.cat((dec_outputs, block_output.unsqueeze(0)), dim=0)

        token_vectors = dec_outputs[-1] @ self.output_embedding
        return token_vectors

    def encoding(self, seq_len, embed_len):
        #PE(pos,2i) = sin(pos/10000 ^ 2i/dmodel )
        #PE(pos,2i+1) = cos(pos/10000 ^ 2i/dmodel
        # Pos 1 will embed to a vector, pos 2 will embed to a vector
        # Adjacent vector positions will form a sin wave
        # plt.plot(np.arange(0,100), np.sin(np.arange(0,100) / np.power(10000.0, (20.0/512))))
        encodings = np.zeros((seq_len, embed_len), dtype=np.float32)
        for i in range(seq_len):
            evens = np.arange(0, embed_len, 2)
            odds = np.arange(1, embed_len, 2)
            all = np.power(np.full(embed_len, 10000), np.arange(0, embed_len) / embed_len)

            sin_embed = np.sin(i/all)
            cos_embed = np.cos(i/all)

            sin_embed[odds] = 0
            cos_embed[evens] = 0
            encodings[i,:] = np.sum((sin_embed, cos_embed), axis=0)
        return encodings

In [361]:
def generate(sequence, pred, target, wrapper):
    prompts = wrapper.decode_batch(sequence.cpu())
    texts = wrapper.decode_batch(torch.argmax(pred, dim=2).cpu())
    correct_texts = wrapper.decode_batch(target.cpu())

    displays = []
    for p, t, ct in zip(prompts, texts, correct_texts):
        displays.append(f"{p} | {ct} | {t}")
    return displays

In [362]:
x, y, prev_y = train.dataset[0]

tf =
decoder_out = tf(x, prev_y)
logits = torch.softmax(decoder_out, dim=1)
generate(x, logits, y, wrapper)

SyntaxError: invalid syntax (2547172622.py, line 3)

In [363]:
from tqdm.auto import tqdm

model = Transformer(wrapper.vocab_size, 512, 8).to(DEVICE)
loss_fn = nn.CrossEntropyLoss(ignore_index=wrapper.pad_token)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [364]:
EPOCHS = 100
DISPLAY_BATCHES = 2

for epoch in range(EPOCHS):
    # Run over the training examples
    train_loss = 0
    match_pct = 0
    for batch, (sequence, target, prev_target) in tqdm(enumerate(train)):
        optimizer.zero_grad()
        pred = model(sequence, prev_target)

        # If you use a batch, need to reshape pred to be batch * sequence, embedding_len to be compatible
        # Similar reshape with target to be batch * sequence vector of class indices
        loss = loss_fn(pred.reshape(-1, pred.shape[-1]), target.view(-1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    with torch.no_grad():
        if epoch % 10 == 0:
            print(f"Epoch {epoch} train loss: {train_loss / len(train)}")
            sents = generate(sequence, pred, target, wrapper)
            for sent in sents:
                print(sent)

466it [00:23, 20.10it/s]


Epoch 0 train loss: 5.84051977922988
"Kristen" identified as aspiring singer | Ashley Youmans, 22 . S | ssss


466it [00:22, 20.83it/s]
466it [00:21, 21.46it/s]
466it [00:21, 21.63it/s]
466it [00:21, 21.60it/s]
466it [00:21, 21.19it/s]
466it [00:22, 21.02it/s]
466it [00:22, 20.87it/s]
466it [00:22, 21.14it/s]
466it [00:21, 21.58it/s]
466it [00:21, 21.73it/s]


Epoch 10 train loss: 1.3104079853375583
The Boeing 777 aircraft first enter | ed service on June 7, 1995 | ed tovvices June 7, 19 s


466it [00:22, 20.94it/s]
466it [00:21, 21.25it/s]
466it [00:21, 21.34it/s]
466it [00:21, 21.37it/s]
466it [00:22, 21.12it/s]
466it [00:21, 21.60it/s]
466it [00:21, 21.30it/s]
466it [00:21, 21.23it/s]
466it [00:22, 20.98it/s]
466it [00:21, 21.36it/s]


...

Epoch 40 train loss: 0.08866790364308547
Boy, 12, lost leg to car bombing in Iraq; | cousin was killed by blast . He is now  | cousin was killed by blast . He is now 


466it [00:21, 21.19it/s]
466it [00:22, 21.18it/s]
466it [00:22, 21.07it/s]
466it [00:21, 21.38it/s]
466it [00:22, 21.11it/s]
466it [00:22, 21.12it/s]
466it [00:22, 21.01it/s]
466it [00:22, 21.00it/s]
466it [00:21, 22.05it/s]
466it [00:20, 22.60it/s]


Epoch 50 train loss: 0.0632246179596617
U.N. to send special envoy | to Myanmar amid reports of crackd | to Myanmardamidds of crackd


466it [00:20, 22.31it/s]
466it [00:21, 21.94it/s]
466it [00:20, 22.54it/s]
466it [00:21, 22.06it/s]
466it [00:20, 23.02it/s]
466it [00:20, 23.03it/s]
182it [00:08, 22.52it/s]


KeyboardInterrupt: 